In [5]:
import tensorflow as tf
import os
import keras
import numpy as np
import easygdf
import mne
import json
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
import get_data

First branch

In [ ]:

def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

input_shape = (1000, 22)

inputs = tf.keras.Input(shape=input_shape)

# First Layer
x = layers.Conv1D(32, 20, strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Second Layer
x = layers.Conv1D(32, 9, strides=2, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
layer1_output = layers.MaxPooling1D()(x)

# Third Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(layer1_output)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)


# Fourth Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.Add()([x, layer1_output])
layer4_output = layers.MaxPooling1D()(x)

# Fifth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(layer4_output)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)


# Sixth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.Add()([x, layer4_output])
x = layers.MaxPooling1D()(x)

x = layers.Dropout(0.5)(x)

x = layers.Flatten()(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.summary()



Second branch

In [ ]:
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

input_shape = (1000, 22)

inputs = tf.keras.Input(shape=input_shape)

# First Layer
x = layers.Conv1D(32, 13, strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)

# Second Layer
x = layers.Conv1D(32, 7, strides=2, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Third Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
layer3_output = layers.MaxPooling1D()(x)

# Fourth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(layer3_output)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)

# Fifth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.Add()([x, layer3_output])
x = layers.MaxPooling1D()(x)

# Dropout layer
x = layers.Dropout(0.5)(x)

# Flatten layer
x = layers.Flatten()(x)

# Output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = models.Model(inputs, outputs)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()



Third branch


In [ ]:
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

input_shape = (1000, 22)

inputs = tf.keras.Input(shape=input_shape)

# First Layer
x = layers.Conv1D(32, 15, strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Second Layer
x = layers.Conv1D(32, 10, strides=2, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)

# Third Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Fourth Layer
x = layers.Conv1D(48, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Dropout layer
x = layers.Dropout(0.5)(x)

# Flatten layer
x = layers.Flatten()(x)

# Output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = models.Model(inputs, outputs)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

the final version

In [4]:
# Define the squeeze excitation block function
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define input shape
input_shape = (1000, 22)

# Define input layers for each branch
input_1 = tf.keras.Input(shape=input_shape)
input_2 = tf.keras.Input(shape=input_shape)
input_3 = tf.keras.Input(shape=input_shape)

# First Branch
x1 = layers.Conv1D(32, 20, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)
x1 = squeeze_excitation_block(x1, r=8)
x1 = layers.MaxPooling1D()(x1)

x1 = layers.Conv1D(32, 9, strides=2, padding='same', kernel_regularizer=l2(0.01))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)
x1 = squeeze_excitation_block(x1, r=8)
x1 = layers.MaxPooling1D()(x1)

x1 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)

x1 = layers.Flatten()(x1)

# Second Branch
x2 = layers.Conv1D(32, 13, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)
x2 = squeeze_excitation_block(x2, r=8)

x2 = layers.Conv1D(32, 7, strides=2, padding='same', kernel_regularizer=l2(0.01))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)
x2 = squeeze_excitation_block(x2, r=8)

x2 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)

x2 = layers.Flatten()(x2)

# Third Branch
x3 = layers.Conv1D(32, 15, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)
x3 = squeeze_excitation_block(x3, r=8)

x3 = layers.Conv1D(32, 10, strides=2, padding='same', kernel_regularizer=l2(0.01))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)
x3 = squeeze_excitation_block(x3, r=8)

x3 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)

x3 = layers.Flatten()(x3)

# Concatenate the outputs of three branches
merged = layers.concatenate([x1, x2, x3])

# Dropout layer
x = layers.Dropout(0.5)(merged)

# # Flatten layer
# x = layers.Flatten()(x)

# # Dense layer
# x = layers.Dense(64, activation='relu')(x)

# Output layer
output = layers.Dense(4, activation='softmax')(x)

# Create model
model = Model(inputs=[input_1, input_2, input_3], outputs=output)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 1000, 22)]           0         []                            
                                                                                                  
 conv1d_9 (Conv1D)           (None, 500, 32)              14112     ['input_4[0][0]']             
                                                                                                  
 batch_normalization_9 (Bat  (None, 500, 32)              128       ['conv1d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 input_5 (InputLayer)        [(None, 1000, 22)]           0         []                        

In [ ]:
events_with_labels_arrays=get_data.event_position()


In [ ]:
data_arrays=get_data.save_data_in_array()

In [28]:

labels = ["A01T", "A02T", "A03T","A04T","A05T","A06T","A07T","A08T","A09T"]

X_train=[]

for label in labels:

    temp_data=data_arrays[label]
    temp_events=events_with_labels_arrays[label]
    for i in range(len(temp_events)):
    
        if temp_events[i,1]==769 or temp_events[i,1]==770 or temp_events[i,1]==771 or temp_events[i,1]==772 :

            ev=temp_events[i,1]
            position=temp_events[i,0]
            for j in range(position, position+1000):

                temp=np.append(temp_data[j,1:23],ev)
                X_train.append(temp)

In [32]:


X_train = np.array(X_train)

#Separate features (X) and labels (y)
X = X_train[:, :-1]  # Extract all rows and all columns except the last one
Y = X_train[:, -1]   # Extract all rows and only the last column

Y = Y.reshape(-1, 1)




[ 0.390625   -4.24804688 -4.05273438  0.9765625  -2.34375    -0.92773438
 -8.05664062 -6.49414062 -6.25       -3.61328125 -5.46875    -5.17578125
 -4.78515625 -5.12695312 -7.27539062 -5.81054688 -7.27539062 -7.27539062
 -5.95703125 -5.71289062 -5.56640625 -2.1484375 ]
